1) For this problem you will use a modified version of the item-based recommender algorithm from Ch. 14 of Machine Learning in Action and use it on joke ratings data based on Jester Online Joke Recommender System. The modified version of the code is provided in the module itemBasedRec.py. Most of the module will be used as is, but you will add some additional functionality.

The data set contains two files. The file "modified_jester_data.csv" contains the ratings on 100 jokes by 1000 users (each row is a user profile). The ratings have been normalized to be between 1 and 21 (a 20-point scale), with 1 being the lowest rating. A zero indicated a missing rating. The file "jokes.csv" contains the joke ids mapped to the actual text of the jokes.

A) Load in the joke ratings data and the joke text data into appropriate data structures.

In [74]:
import numpy as np
import pandas as pd
from numpy import linalg as la
from numpy import *

In [47]:
# Loading in joke ratings data
joke_ratings = np.genfromtxt('jokes/modified_jester_data.csv',delimiter=',')
joke_ratings

array([[ 3.18, 19.79,  1.34, ...,  0.  ,  0.  ,  0.  ],
       [15.08, 10.71, 17.36, ..., 11.34,  6.68, 12.07],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       ...,
       [16.58, 16.63, 15.85, ...,  0.  ,  0.  ,  0.  ],
       [ 3.67,  4.45,  3.67, ...,  3.77,  3.77,  3.28],
       [ 9.88, 11.73,  9.16, ...,  0.  ,  0.  ,  0.  ]])

Implementing methods in the itemBasedRec.py file

In [48]:
def ecludSim(inA,inB):
    return 1.0 / (1.0 + la.norm(inA - inB))

In [49]:
def pearsSim(inA,inB):
    if len(inA) < 3 : return 1.0
    return 0.5 + 0.5 * corrcoef(inA, inB, rowvar = 0)[0][1]

In [50]:
def cosSim(inA,inB):
    num = float(inA.T * inB)
    denom = la.norm(inA)*la.norm(inB)
    return 0.5 + 0.5 * (num / denom)

In [51]:
def standEst(dataMat, user, simMeas, item):
    n = shape(dataMat)[1]
    simTotal = 0.0; ratSimTotal = 0.0
    for j in range(n):
        userRating = dataMat[user,j]
        if userRating == 0: continue
        overLap = nonzero(logical_and(dataMat[:,item]>0, \
                                      dataMat[:,j]>0))[0]
        if len(overLap) == 0: similarity = 0
        else: similarity = simMeas(dataMat[overLap,item], \
                                   dataMat[overLap,j])
        #print 'the %d and %d similarity is: %f' % (item, j, similarity)
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0: return 0
    else: return ratSimTotal/simTotal

In [52]:
def svdEst(dataMat, user, simMeas, item):
    n = shape(dataMat)[1]
    simTotal = 0.0; ratSimTotal = 0.0
    data=mat(dataMat)
    U,Sigma,VT = la.svd(data)
    Sig4 = mat(eye(4)*Sigma[:4]) #arrange Sig4 into a diagonal matrix
    xformedItems = data.T * U[:,:4] * Sig4.I  #create transformed items
    for j in range(n):
        userRating = data[user,j]
        if userRating == 0 or j==item: continue
        similarity = simMeas(xformedItems[item,:].T,\
                             xformedItems[j,:].T)
        #print 'the %d and %d similarity is: %f' % (item, j, similarity)
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0: return 0
    else: return ratSimTotal/simTotal

In [53]:
# This function is not needed for Assignment 4, but may be useful for experimentation
def recommend(dataMat, user, N=3, simMeas=cosSim, estMethod=standEst):
    unratedItems = nonzero(dataMat[user,:].A==0)[1] #find unrated items 
    if len(unratedItems) == 0: return 'you rated everything'
    itemScores = []
    for item in unratedItems:
        estimatedScore = estMethod(dataMat, user, simMeas, item)
        itemScores.append((item, estimatedScore))
    return sorted(itemScores, key=lambda jj: jj[1], reverse=True)[:N]

In [97]:
# This function performs evaluatoin on a single user based on the test_ratio
# For example, with test_ratio = 0.2, a randomly selected 20 percent of rated 
# items by the user are withheld and the rest are used to estimate the withheld ratings

def cross_validate_user(dataMat, user, test_ratio, estMethod=standEst, simMeas=pearsSim):
    number_of_items = np.shape(dataMat)[1]
    rated_items_by_user = np.array([i for i in range(number_of_items) if dataMat[user,i]>0])
    test_size = test_ratio * len(rated_items_by_user)
    test_indices = np.random.randint(0, len(rated_items_by_user), int(test_size))
    withheld_items = rated_items_by_user[test_indices]
    original_user_profile = np.copy(dataMat[user])
    dataMat[user, withheld_items] = 0 # So that the withheld test items is not used in the rating estimation below
    error_u = 0.0
    count_u = len(withheld_items)

    # Compute absolute error for user u over all test items
    for item in withheld_items:
        # Estimate rating on the withheld item
        estimatedScore = estMethod(dataMat, user, simMeas, item)
        error_u = error_u + abs(estimatedScore - original_user_profile[item])

    # Now restore ratings of the withheld items to the user profile
    for item in withheld_items:
        dataMat[user, item] = original_user_profile[item]

    # Return sum of absolute errors and the count of test cases for this user
    # Note that these will have to be accumulated for each user to compute MAE
    return error_u, count_u

In [82]:
def load_jokes(file):
    jokes = np.genfromtxt(file, delimiter=',', dtype=str)
    jokes = np.array(jokes[:,1])
    return jokes

In [83]:
def get_joke_text(jokes, id):
    return jokes[id]

In [84]:
# Loading in Joke text data
jokes = load_jokes('jokes/jokes.csv')
jokes.shape

(100,)

B) Complete the definition for the function "test". This function iterates over all users and for each performs evaluation (by calling the provided "cross_validate_user" function), and returns the error information necessary to compute Mean Absolute Error (MAE). Use this function to perform evaluation (wiht 20% test-ratio for each user) comparing MAE results using standard item-based collaborative filtering (based on the rating prediction function "standEst") with results using the SVD-based version of the rating item-based CF (using "svdEst" as the prediction engine).

In [92]:
# Completing definition for the test function
def test(dataMat, test_ratio, estMethod):
    # Write this function to iterate over all users and for each perform evaluation by calling
    # the above cross_validate_user function on each user. MAE will be the ratio of total error 
    # across all test cases to the total number of test cases, for all users
    
    total_error = 0
    total_count = 0

    # Iterate through 
    for i in range(len(dataMat)):
        error_u, count_u = cross_validate_user(dataMat, i, test_ratio, estMethod=standEst, simMeas=pearsSim)
        total_error += error_u
        total_count += count_u
    
    MAE = total_error/total_count
        
    print('Mean Absoloute Error for ',estMethod,' : ', MAE)

In [99]:
# Using standEst
mae_stand = test(joke_ratings, 0.2, standEst)
mae_stand

Mean Absoloute Error for  <function standEst at 0x000002C129EDDF70>  :  3.7446586735294702


In [100]:
# USing svdEst
mae_svd = test(joke_ratings, 0.2, svdEst)
mae_svd

Mean Absoloute Error for  <function svdEst at 0x000002C129EC3280>  :  3.69551341428179


Using standEst will create a higher mean absolute error than using svdEst. The difference is not by much but it is still important to note the difference in prediction engines.

C) Write a new function "print_most_similar_jokes" which takes the joke ratings data, a query joke id, a parameter k for the number of nearest neighbors, and a similarity metric function, and prints the text of the query joke as well as the texts of the top k most similar jokes based on user ratings.

In [109]:
def print_most_similar_jokes(dataMat, jokes, queryJoke, k, metric=pearsSim):
    # Write this function to find the k most similar jokes (based on user ratings) to a queryJoke
    # The queryJoke is a joke id as given in the 'jokes.csv' file (an corresponding to the a column in dataMat)
    # You must compare ratings for the queryJoke (the column in dataMat corresponding to the joke), to all
    # other joke rating vectors and return the top k. Note that this is the same as performing KNN on the 
    # columns of dataMat. The function must retrieve the text of the joke from 'jokes.csv' file and print both
    # the queryJoke text as well as the text of the returned jokes.
    
    # Get the joke from the jokes data based on the queryJoke
    queryJoke_data = dataMat[:,queryJoke]
    print("Joke: ", jokes[queryJoke], "\n")
    
    # Store the similar jokes 
    similar = []
    for i in range(dataMat.shape[1]):
        # Get similarity of the joke and store it in the list
        v = dataMat[:,i]
        s = metric(queryJoke_data, v)
        similar.append(s)
    
    # Sort the list based on similarity
    similar = np.array(similar)
    similar = similar.argsort()
    knn = zeros((k, dataMat.shape[1]))
    
    # Store the top k jokes
    top = []
    for i in range(k):
        # Add the top k jokes to the list
        knn[i,:] = dataMat[similar[i],:]
        top.append(similar[i])
    
    # print the top k jokes
    print("Top %d Similar Jokes:" %k)
    for i in top:
        print(jokes[i], "\n")

In [111]:
print_most_similar_jokes(joke_ratings, jokes, 1, 3, pearsSim)

Joke:  This couple had an excellent relationship going until one day he came home from work to find his girlfriend packing. He asked her why she was leaving him and she told him that she had heard awful things about him. "What could they possibly have said to make you move out?" "They told me that you were a pedophile." He replied "That's an awfully big word for a ten year old." 

Top 3 Similar Jokes:
Q. What is orange and sounds like a parrot?  A. A carrot. 

Q: If a person who speaks three languages is called "tri-lingual" and a person who speaks two languages is called "bi-lingual" what do calla person who only speaks one language?A: American! 

Q. What's O. J. Simpson's Internet address? A.	Slash slash backslash slash slash escape. 



In [112]:
print_most_similar_jokes(joke_ratings, jokes, 4, 5, pearsSim)

Joke:  Q. What's O. J. Simpson's Internet address? A.	Slash slash backslash slash slash escape. 

Top 5 Similar Jokes:
A man visits the doctor. The doctor says "I have bad news for you.You have cancer and Alzheimer's disease". The man replies "Well thank God I don't have cancer!" 

This couple had an excellent relationship going until one day he came home from work to find his girlfriend packing. He asked her why she was leaving him and she told him that she had heard awful things about him. "What could they possibly have said to make you move out?" "They told me that you were a pedophile." He replied "That's an awfully big word for a ten year old." 

What did the Buddhist say to the hot dog vendor?Make me one with everything. 

A teacher is explaining to her class how different languages use negatives differently.  She says "In all languages a positive followed by a negative or a negative followed by a positive makes a negative.  In some languages two negatives together make a positiv

In [113]:
print_most_similar_jokes(joke_ratings, jokes, 10, 7, pearsSim)

Joke:  Q. What do a hurricane a tornado and a redneck divorce all have in common? A. Someone's going to lose their trailer... 

Top 7 Similar Jokes:
Q. What is orange and sounds like a parrot?  A. A carrot. 

Q:  What did the blind person say when given some matzah?A:  Who the hell wrote this? 

A dog walks into Western Union and asks the clerk to send a telegram. He fills out a form on which he writes down the telegram he wishes to send: "Bow wow wow Bow wow wow."The clerk says "You can add another 'Bow wow' for the same price."The dog responded "Now wouldn't that sound a little silly?" 

On the first day of college the Dean addressed the students pointing out some of the rules:"The female dormitory will be out-of-bounds for all male students and the male dormitory to the female students. Anybody caught breaking this rule will be fined $20 the first time." He continued "Anybody caught breaking this rule the second time will be fined $60. Being caught a third time will cost you a fine 